# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that you've practiced regularization, initialization, and optimization techniques, its time to synthesize these concepts into a cohesive modeling pipeline.  

With this pipeline, you will not only fit an initial model but also attempt to improve it. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modeling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. You've seen the bias variance trade-off, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Apply normalization as a preprocessing technique 
* Implement a K-folds cross validation modeling pipeline for deep learning models 
* Apply regularization techniques to improve your model's performance 

## Load the data

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from keras import models
from keras import layers
from keras import regularizers
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


In this lab you'll be working with the *The Lending Club* data. 

- Import the data available in the file `'loan_final.csv'` 
- Drop rows with missing values in the `'total_pymnt'` column (this is your target column) 
- Print the first five rows of the data 
- Print the dimensions of the data 

In [3]:
# Import the data
data = pd.read_csv('loan_final.csv', header = 0)

# Drop rows with no target value
data.dropna(subset = ['total_pymnt'], inplace = True)

# Print the first five rows
data.head(5)

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [4]:
# Print the dimensions of data 
data.shape

(42535, 16)

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modeling decisions. As such, pull out a sample of 30% of the total available data. For consistency of results, use random seed 42. 

In [5]:
# Features to build the model
features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc', 
            'home_ownership', 'verification_status', 'emp_length']

X = data[features]
y = data[['total_pymnt']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

## Preprocessing (Numerical features) 

- Fill all missing values in numeric features with their respective means 
- Standardize all the numeric features  
- Convert the final results into DataFrames 

In [6]:
# Select continuous features
cont_features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']

X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Instantiate SimpleImputer - fill the missing values with the mean
si = SimpleImputer()

# Fit and transform the training data
X_train_imputed = si.fit_transform(X_train_cont)

# Transform test data
X_test_imputed = si.transform(X_test_cont)

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform the training data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_imputed), columns= cont_features)

# Transform test data
X_test_scaled = pd.DataFrame(ss_X.fit_transform(X_test_imputed), columns=cont_features)

## Preprocessing (Categorical features) 

- Fill all missing values in categorical features with the string `'missing'` 
- One-hot encode all categorical features 
- Convert the final results into DataFrames 


In [9]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]
X_test_cat = X_test.loc[:, cat_features]

# Fill missing values with the string 'missing'
X_train_cat.fillna(value = 'missing', inplace = True)
X_test_cat.fillna(value = 'missing', inplace = True)



# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Get all categorical feature names
cat_columns = ohe.get_feature_names(input_features=X_train_cat.columns)

# Fit and transform the training data
X_train_categorical = pd.DataFrame(X_train_ohe.todense(), columns=cat_columns)

# Transform test data
X_test_categorical = pd.DataFrame(X_test_ohe.todense(), columns=cat_columns)

Run the below cell to combine the numeric and categorical features. 

In [10]:
# Combine continuous and categorical feature DataFrames
X_train_all = pd.concat([X_train_scaled, X_train_categorical], axis=1)
X_test_all = pd.concat([X_test_scaled, X_test_categorical], axis=1)

# Number of input features
n_features = X_train_all.shape[1]

- Standardize the target DataFrames (`y_train` and `y_test`) 

In [11]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform Y (train)
y_train_scaled = ss_y.fit_transform(y_train)

# Transform test Y (test)
y_test_scaled = ss_y.transform(y_test)

## Define a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, you will perform k-fold cross-validation using the following steps: 

- Create a function that returns a compiled deep learning model 
- Use the wrapper function `KerasRegressor()` that defines how these folds are trained 
- Call the `cross_val_predict()` function to perform k-fold cross-validation 

In the cell below, we've defined a baseline model that returns a compiled Keras models. 

In [12]:
# Define a function that returns a compiled Keras model 
def create_baseline_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

Wrap `create_baseline_model` inside a call to `KerasRegressor()`, and: 

- Train for 150 epochs 
- Set the batch size to 256 

> NOTE: Refer to the [documentation](https://keras.io/scikit-learn-api/) to learn about `KerasRegressor()`.  

In [13]:
# Wrap the above function for use in cross-validation
keras_wrapper_1 = KerasRegressor(create_baseline_model, epochs = 150, batch_size = 256)

Use `cross_val_predict()` to generate cross-validated predictions with: 
- 5-fold cv 
- scaled input (`X_train_all`) and output (`y_train_scaled`) 

In [14]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_baseline_preds = cross_val_predict(keras_wrapper_1, X_train_all, y_train_scaled, cv = 5)

Epoch 1/150
23819/23819 [==============================] - 1s 24us/step - loss: 0.5847 - mse: 0.5847
Epoch 2/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2697 - mse: 0.2697
Epoch 3/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2476 - mse: 0.2476
Epoch 4/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2360 - mse: 0.2360
Epoch 5/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2286 - mse: 0.2286
Epoch 6/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2238 - mse: 0.2238
Epoch 7/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2200 - mse: 0.2200
Epoch 8/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2173 - mse: 0.2173
Epoch 9/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2151 - mse: 0.2151
Epoch 10/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2135 - m

23819/23819 [==============================] - 0s 14us/step - loss: 0.1975 - mse: 0.1975
Epoch 82/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.1974 - mse: 0.1974
Epoch 83/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1974 - mse: 0.1974
Epoch 84/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1974 - mse: 0.1974
Epoch 85/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1974 - mse: 0.1974
Epoch 86/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1968 - mse: 0.1968
Epoch 87/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1972 - mse: 0.1972
Epoch 88/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1971 - mse: 0.1971
Epoch 89/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1971 - mse: 0.1971
Epoch 90/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1968 - mse: 

23819/23819 [==============================] - 0s 10us/step - loss: 0.2233 - mse: 0.2233
Epoch 12/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.2227 - mse: 0.2227
Epoch 13/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.2211 - mse: 0.2211
Epoch 14/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2198 - mse: 0.2198
Epoch 15/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2186 - mse: 0.2186
Epoch 16/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2176 - mse: 0.2176
Epoch 17/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.2168 - mse: 0.2168
Epoch 18/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.2160 - mse: 0.2160
Epoch 19/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2152 - mse: 0.2152
Epoch 20/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2145 - mse: 0.214

23819/23819 [==============================] - 0s 10us/step - loss: 0.2009 - mse: 0.2009
Epoch 92/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2010 - mse: 0.2010
Epoch 93/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2008 - mse: 0.2008
Epoch 94/150
23819/23819 [==============================] - 0s 16us/step - loss: 0.2009 - mse: 0.2009
Epoch 95/150
23819/23819 [==============================] - 0s 15us/step - loss: 0.2006 - mse: 0.2006
Epoch 96/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.2004 - mse: 0.2004
Epoch 97/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.2007 - mse: 0.2007
Epoch 98/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2006 - mse: 0.2006
Epoch 99/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2003 - mse: 0.2003
Epoch 100/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2005 - mse: 0

23819/23819 [==============================] - 0s 14us/step - loss: 0.2059 - mse: 0.2059
Epoch 22/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2054 - mse: 0.2054
Epoch 23/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2049 - mse: 0.2049
Epoch 24/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.2047 - mse: 0.2047
Epoch 25/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2045 - mse: 0.2045
Epoch 26/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.2039 - mse: 0.2039
Epoch 27/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.2035 - mse: 0.2035
Epoch 28/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2029 - mse: 0.2029
Epoch 29/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2030 - mse: 0.2030
Epoch 30/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2026 - mse: 

23819/23819 [==============================] - 0s 12us/step - loss: 0.1953 - mse: 0.1953
Epoch 102/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.1949 - mse: 0.1949
Epoch 103/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.1948 - mse: 0.1948
Epoch 104/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.1947 - mse: 0.1947
Epoch 105/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.1951 - mse: 0.1951
Epoch 106/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1950 - mse: 0.1950
Epoch 107/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.1947 - mse: 0.1947
Epoch 108/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.1944 - mse: 0.1944
Epoch 109/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.1946 - mse: 0.1946
Epoch 110/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.194

23819/23819 [==============================] - 0s 14us/step - loss: 0.2070 - mse: 0.2070
Epoch 32/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.2068 - mse: 0.2068
Epoch 33/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.2066 - mse: 0.2066
Epoch 34/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.2063 - mse: 0.2063
Epoch 35/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.2061 - mse: 0.2061
Epoch 36/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.2074 - mse: 0.2074
Epoch 37/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.2054 - mse: 0.2054
Epoch 38/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.2055 - mse: 0.2055
Epoch 39/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.2052 - mse: 0.2052
Epoch 40/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.2049 - mse: 

23819/23819 [==============================] - 0s 13us/step - loss: 0.1976 - mse: 0.1976
Epoch 112/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1978 - mse: 0.1978
Epoch 113/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1975 - mse: 0.1975
Epoch 114/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1973 - mse: 0.1973
Epoch 115/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.1973 - mse: 0.1973
Epoch 116/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1972 - mse: 0.1972
Epoch 117/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1970 - mse: 0.1970
Epoch 118/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1972 - mse: 0.1972
Epoch 119/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.1969 - mse: 0.1969
Epoch 120/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.196

23820/23820 [==============================] - 0s 14us/step - loss: 0.2033 - mse: 0.2033
Epoch 42/150
23820/23820 [==============================] - 0s 12us/step - loss: 0.2031 - mse: 0.2031
Epoch 43/150
23820/23820 [==============================] - 0s 12us/step - loss: 0.2029 - mse: 0.2029
Epoch 44/150
23820/23820 [==============================] - 0s 13us/step - loss: 0.2029 - mse: 0.2029
Epoch 45/150
23820/23820 [==============================] - 0s 12us/step - loss: 0.2025 - mse: 0.2025
Epoch 46/150
23820/23820 [==============================] - 0s 17us/step - loss: 0.2023 - mse: 0.2023
Epoch 47/150
23820/23820 [==============================] - 0s 12us/step - loss: 0.2022 - mse: 0.2022
Epoch 48/150
23820/23820 [==============================] - 0s 13us/step - loss: 0.2020 - mse: 0.2020
Epoch 49/150
23820/23820 [==============================] - 0s 12us/step - loss: 0.2019 - mse: 0.2020
Epoch 50/150
23820/23820 [==============================] - ETA: 0s - loss: 0.2025 - mse: 0.202

23820/23820 [==============================] - 0s 12us/step - loss: 0.1955 - mse: 0.1955
Epoch 121/150
23820/23820 [==============================] - 0s 12us/step - loss: 0.1954 - mse: 0.1954
Epoch 122/150
23820/23820 [==============================] - 0s 13us/step - loss: 0.1951 - mse: 0.1951
Epoch 123/150
23820/23820 [==============================] - 0s 14us/step - loss: 0.1953 - mse: 0.1953
Epoch 124/150
23820/23820 [==============================] - 0s 15us/step - loss: 0.1955 - mse: 0.1955
Epoch 125/150
23820/23820 [==============================] - 0s 14us/step - loss: 0.1951 - mse: 0.1951
Epoch 126/150
23820/23820 [==============================] - 0s 14us/step - loss: 0.1953 - mse: 0.1953
Epoch 127/150
23820/23820 [==============================] - 0s 16us/step - loss: 0.1948 - mse: 0.1948
Epoch 128/150
23820/23820 [==============================] - 0s 14us/step - loss: 0.1948 - mse: 0.1948
Epoch 129/150
23820/23820 [==============================] - 0s 12us/step - loss: 0.195

- Find the RMSE on train data 

In [16]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_baseline_preds))

0.44420065108471557

- Convert the scaled predictions back to original scale 
- Calculate RMSE in the original units with `y_train` and `baseline_preds` 

In [17]:
# Convert the predictions back to original scale
baseline_preds = ss_y.inverse_transform(cv_baseline_preds)

# RMSE on train data (original scale)
np.sqrt(mean_squared_error(y_train_scaled, baseline_preds))

14417.747586684942

## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.

In [30]:
# Define a function that returns a compiled Keras model 
def create_bigger_model():
    
    bigger_data_model = models.Sequential()
    bigger_data_model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))
    bigger_data_model.add(layers.Dense(10, activation='relu'))
    bigger_data_model.add(layers.Dense(10, activation='relu'))
    bigger_data_model.add(layers.Dense(10, activation='relu'))
    bigger_data_model.add(layers.Dense(1, activation='softmax'))

    bigger_data_model.compile(optimizer='SGD', 
                          loss='mse', 
                          metrics=['mse'])

    return bigger_data_model

In [31]:
# Wrap the above function for use in cross-validation
keras_wrapper_2 =KerasRegressor(create_bigger_model,  
                                 epochs=150, 
                                 batch_size=256, 
                                 verbose=0)

In [32]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_bigger_model_preds = cross_val_predict(keras_wrapper_2, X_train_all, y_train_scaled, cv=5)

In [33]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_bigger_model_preds))

1.4142135623730951

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least two different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting. 

In [34]:
# Define a function that returns a compiled Keras model 
def create_regularized_model():
    
    dropout_model = models.Sequential()

# Implement dropout to the input layer
# NOTE: This is where you define the number of units in the input layer
    dropout_model.add(layers.Dropout(0.3, input_shape=(n_features,)))

# Add the first hidden layer
    dropout_model.add(layers.Dense(10, activation='relu'))

# Implement dropout to the first hidden layer 
    dropout_model.add(layers.Dropout(0.3))

# Add the second hidden layer
    dropout_model.add(layers.Dense(10, activation='relu'))

# Implement dropout to the second hidden layer 
    dropout_model.add(layers.Dropout(0.3))

# Add the output layer
    dropout_model.add(layers.Dense(1, activation='softmax'))


# Compile the model
    dropout_model.compile(optimizer='SGD', 
                      loss='mse', 
                      metrics=['mse'])
    return dropout_model

In [35]:
# Wrap the above function for use in cross-validation
keras_wrapper_3 = KerasRegressor(create_regularized_model,  
                                 epochs=150, 
                                 batch_size=256, 
                                 verbose=0)

In [36]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_dropout_preds = cross_val_predict(keras_wrapper_3, X_train_all, y_train_scaled, cv=5)

In [37]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_dropout_preds))

1.4142135623730951

In [40]:
def create_L2_model():
    L2_model = models.Sequential()

# Add the input and first hidden layer
    L2_model.add(layers.Dense(15, activation='relu', kernel_regularizer=regularizers.l2(0.005), input_shape = (n_features,)))
    L2_model.add(layers.Dense(10, activation='relu', kernel_regularizer=regularizers.l2(0.005)))

# Add another hidden layer


# Add an output layer
    L2_model.add(layers.Dense(1, activation='softmax'))

# Compile the model
    L2_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])
    return L2_model

In [42]:
keras_wrapper_4 = KerasRegressor(create_L2_model,  
                                 epochs=150, 
                                 batch_size=256, 
                                 verbose=0)

In [43]:
np.random.seed(123)
cv_l2_preds = cross_val_predict(keras_wrapper_4, X_train_all, y_train_scaled, cv=5)

In [44]:
np.sqrt(mean_squared_error(y_train_scaled, cv_l2_preds))

1.4142135623730951

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your final model on the test set. Fit the model using all of the training data using the architecture and hyperparameters that were most effective in your experiments above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modeling process)! 

In [45]:
# ⏰ This cell may take several mintes to run
model = models.Sequential()

# Implement dropout to the input layer
# NOTE: This is where you define the number of units in the input layer
model.add(layers.Dropout(0.3, input_shape=(n_features,)))

# Add the first hidden layer
model.add(layers.Dense(10, activation='relu'))

# Implement dropout to the first hidden layer 
model.add(layers.Dropout(0.3))

# Add the second hidden layer
model.add(layers.Dense(10, activation='relu'))

# Implement dropout to the second hidden layer 
model.add(layers.Dropout(0.3))

# Add the output layer
model.add(layers.Dense(1, activation='softmax'))


# Compile the model
model.compile(optimizer='SGD', 
                      loss='mse', 
                      metrics=['mse'])

In [46]:
model.fit(X_train_all, 
          y_train_scaled, 
          epochs=150, 
          batch_size=256, 
          verbose=0)

In [47]:
final_preds_scaled = model.predict(X_test_all)

# Convert the predictions back to original scale 
final_preds = ss_y.inverse_transform(final_preds_scaled)

# RMSE on test data (original scale)
np.sqrt(mean_squared_error(y_test, final_preds))

12839.966701846191

## Summary

In this lab, you investigated some data from *The Lending Club* in a complete data science pipeline to build neural networks with good performance. You began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, you implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. 